In [20]:
import strawberryfields as sf
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from scipy.stats import skew, kurtosis
from itertools import product
import multiprocessing as mp
import json
import os
import seaborn as sns
from tqdm import tqdm   # <-- Added import for progress bar

# ==============================================================================
# CONFIGURATION: ALL PARAMETERS USED IN THE PIPELINE
# ==============================================================================
PIPELINE_CONFIG = {
    "sf_hbar": 1,
    "quad_range": (-6, 6),
    "quad_points": 200, 
    "num_run_per_sample": 5,    
    "num_shots": 500,    # homodyne measurements per grid point (not used for these marginal features)
    "grid": {
        # "num_db": 3,       # distinct squeezing values (in dB)
        "num_loss": 3,     # distinct loss values
        "num_gamma": 3     # distinct dephasing values
    },
    # "db_range": (10.0, 13.0),   # dB values range
    "loss_range": (0.9, 1.0),     # this actually transimitivity
    "gamma_range": (0.0, 0.2),    # dephasing values range
    "model": {
        "input_shape": (8,),        
        "loss_bins": 20,
        "dephasing_bins": 20,
        "learning_rate": 1e-3,
        "epochs": 1000,
        "batch_size": 32,
        "validation_split": 0
    },
    "save_paths": {
        "dataset": "dataset.npz",
        "model": "trained_model.h5",
        "config": "pipeline_config.json"
    }
}

# Save the pipeline configuration to disk.
with open(PIPELINE_CONFIG["save_paths"]["config"], "w") as f:
    json.dump(PIPELINE_CONFIG, f, indent=4)

# ==============================================================================
# SETUP: CONFIGURATION PARAMETERS AND GLOBAL VARIABLES
# ==============================================================================
print("Step 1: Setting configuration parameters.")
CONFIG = {
    "hbar": PIPELINE_CONFIG["sf_hbar"],
    "quad_range": PIPELINE_CONFIG["quad_range"],
    "quad_points": PIPELINE_CONFIG["quad_points"],
    "cmap": plt.cm.RdBu,
    "colors": {
        "q_marginal": "navy",
        "p_marginal": "maroon",
        "samples_x": "skyblue",
        "samples_p": "lightcoral",
        "ideal": "red",
        "noisy": "green"
    }
}
sf.hbar = CONFIG["hbar"]
CONFIG["scale"] = np.sqrt(CONFIG["hbar"] * np.pi)
print("Configuration set. Scale =", CONFIG["scale"])

# ==============================================================================
# UTILITY: dB to epsilon conversion and feature extraction
# ==============================================================================
def db_to_epsilon(db_val: float) -> float:
    """
    Convert a given GKP squeezing level in dB to epsilon via:
         tanh(epsilon) = 10^(-db_val/10)
    """
    t = 10.0 ** (-db_val / 10.0)
    eps = 0.5 * np.log((1.0 + t) / (1.0 - t))
    return eps

def extract_statistical_features(samples_x: np.ndarray, samples_p: np.ndarray) -> np.ndarray:
    """Extract mean, variance, skewness, and kurtosis for X and P quadratures."""
    x_features = [
        np.mean(samples_x), np.var(samples_x),
        skew(samples_x), kurtosis(samples_x)
    ]
    p_features = [
        np.mean(samples_p), np.var(samples_p),
        skew(samples_p), kurtosis(samples_p)
    ]
    return np.array(x_features + p_features)

def extract_histogram_features(
    samples_x: np.ndarray, 
    samples_p: np.ndarray, 
    bins: int = 100, 
    range_val: Tuple[float, float] = (-6, 6)
) -> np.ndarray:
    """
    Compute histograms for X and P quadrature samples and return a concatenated feature vector.
    The histograms are computed with density=True so that they approximate a probability density.
    """
    hist_x, _ = np.histogram(samples_x, bins=bins, density=True, range=range_val)
    hist_p, _ = np.histogram(samples_p, bins=bins, density=True, range=range_val)
    return np.concatenate([hist_x, hist_p])

Step 1: Setting configuration parameters.
Configuration set. Scale = 1.7724538509055159


In [21]:
# ==============================================================================
# CORE SIMULATION FUNCTIONS
# ==============================================================================

def simulate_homodyne(prep_state: List[float], epsilon: float, noise_params: Dict, num_samples: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Simulate homodyne measurements (X and P quadratures) for a given epsilon and noise parameters.
    Returns two arrays of measurement outcomes.
    """
    samples_x, samples_p = [], []
    for _ in range(num_samples):
        prog_x = sf.Program(1)
        with prog_x.context as q:
            sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
            if 'loss' in noise_params:
                sf.ops.LossChannel(noise_params['loss']) | q
            if 'gamma' in noise_params:
                theta = np.random.normal(0, np.sqrt(2 * noise_params['gamma']))
                sf.ops.Rgate(theta) | q
            sf.ops.MeasureX | q
        eng_x = sf.Engine("bosonic")
        sample_x = eng_x.run(prog_x).samples[0, 0] / CONFIG["scale"]
        samples_x.append(sample_x)
        
        prog_p = sf.Program(1)
        with prog_p.context as q:
            sf.ops.GKP(state=prep_state, epsilon=epsilon) | q
            if 'loss' in noise_params:
                sf.ops.LossChannel(noise_params['loss']) | q
            if 'gamma' in noise_params:
                theta = np.random.normal(0, np.sqrt(2 * noise_params['gamma']))
                sf.ops.Rgate(theta) | q
            sf.ops.MeasureP | q
        eng_p = sf.Engine("bosonic")
        sample_p = eng_p.run(prog_p).samples[0, 0] / CONFIG["scale"]
        samples_p.append(sample_p)
    
    return np.array(samples_x), np.array(samples_p)

def plot_quadratures(
    samples_x: np.ndarray,
    samples_p: np.ndarray,
    noisy_x: np.ndarray,
    noisy_p: np.ndarray,
    title: str = "",
    figsize: Tuple[float, float] = (12, 5)
):
    """Plot quadrature measurement results"""
    quad_axis = np.linspace(-6, 6, 1000)
    fig, axs = plt.subplots(1, 2, figsize=figsize)
    
    # X quadrature
    x_hist, x_bin_edges, _ = axs[0].hist(samples_x, bins=100, density=True, 
               color=CONFIG["colors"]["samples_x"], label="Samples")
    # axs[0].plot(quad_axis, ideal_x, '--', color=CONFIG["colors"]["ideal"], label="Ideal")
    axs[0].plot(quad_axis, noisy_x, '-', color=CONFIG["colors"]["noisy"], label="Noisy")
    axs[0].set_xlabel(r"$q$ ($\sqrt{\pi\hbar}$)", fontsize=12)
    axs[0].set_ylabel("Probability Density", fontsize=12)

    # Add bin overlays for X quadrature
    for j in range(-3, 4):
        axs[0].axvspan(2*j - 0.5, 2*j + 0.5, alpha=0.2, facecolor='b')
        axs[0].axvspan(2*j + 0.5, 2*j + 1.5, alpha=0.2, facecolor='r')
    
    # P quadrature
    p_hist, p_bin_edges, _ = axs[1].hist(samples_p, bins=100, density=True, 
               color=CONFIG["colors"]["samples_p"], label="Samples")
    # axs[1].plot(quad_axis, ideal_p, '--', color=CONFIG["colors"]["ideal"], label="Ideal")
    axs[1].plot(quad_axis, noisy_p, '-', color=CONFIG["colors"]["noisy"], label="Noisy")
    axs[1].set_xlabel(r"$p$ ($\sqrt{\pi\hbar}$)", fontsize=12)

    # Add bin overlays for P quadrature
    for j in range(-3, 4):
        axs[1].axvspan(2*j - 0.5, 2*j + 0.5, alpha=0.2, facecolor='b')
        axs[1].axvspan(2*j + 0.5, 2*j + 1.5, alpha=0.2, facecolor='r')
    
    for ax in axs:
        ax.legend()
        ax.tick_params(axis='both', which='major', labelsize=10)
    
    if title:
        fig.suptitle(title, y=1.02)
    plt.tight_layout()
    plt.show()

    return (x_hist, x_bin_edges), (p_hist, p_bin_edges)

In [ ]:
print("Step 3: Preparing grid parameters for dataset generation.")

grid_config = PIPELINE_CONFIG["grid"]
num_loss = grid_config["num_loss"]
num_gamma = grid_config["num_gamma"]

# Define the parameter ranges.
loss_values = np.linspace(PIPELINE_CONFIG["loss_range"][0], PIPELINE_CONFIG["loss_range"][1], num_loss)
gamma_values = np.linspace(PIPELINE_CONFIG["gamma_range"][0], PIPELINE_CONFIG["gamma_range"][1], num_gamma)

print(loss_values)
print(gamma_values)

# Define the GKP preparation states and dB values.
prop_states = [[0, 0]]
db = [10.0]

# Create a grid over all parameters: (prep_state, db, loss, gamma)
grid_params = list(product(prop_states, db, loss_values, gamma_values))

def process_grid_point(params):
    prep_state, db_val, loss_val, gamma_val = params
    epsilon = db_to_epsilon(db_val)
    current_noise_params = {"loss": loss_val, "gamma": gamma_val}

    # Run the simulation multiple times for this grid point.
    features_all = []
    for _ in range(PIPELINE_CONFIG["num_run_per_sample"]):
        samples_x, samples_p = simulate_homodyne(
            prep_state=prep_state,
            epsilon=epsilon,
            noise_params=current_noise_params,
            num_samples=PIPELINE_CONFIG["num_shots"]
        )
       
        features = extract_histogram_features(samples_x, samples_p, bins=100, range_val=(-6, 6))
        features_all.append(features)
    
    # Map the (loss, gamma) to discrete indices for classification.
    loss_idx = np.where(np.isclose(loss_values, loss_val))[0][0]
    dephasing_idx = np.where(np.isclose(gamma_values, gamma_val))[0][0]
    return features_all, loss_idx, dephasing_idx

dataset_path = PIPELINE_CONFIG["save_paths"]["dataset"]
if not os.path.exists(dataset_path):
    print("No dataset found. Generating new dataset...")
    results = []
    for params in tqdm(grid_params, total=len(grid_params)):
        results.append(process_grid_point(params))

    X_data_list = []
    y_joint_list = []
    for features_list, loss_idx, dephasing_idx in results:
        joint_label = loss_idx * num_gamma + dephasing_idx
        for features in features_list:
            X_data_list.append(features)
            y_joint_list.append(joint_label)
    
    X_data = np.array(X_data_list)
    y_joint = np.array(y_joint_list)
   
    # Save the dataset.
    np.savez_compressed(PIPELINE_CONFIG["save_paths"]["dataset"],
                        X_data=X_data, y_joint=y_joint)
    print("Dataset saved to", PIPELINE_CONFIG["save_paths"]["dataset"])
else:
    data = np.load(PIPELINE_CONFIG["save_paths"]["dataset"], allow_pickle=True)
    X_data = data['X_data']
    y_joint = data['y_joint']

Step 3: Preparing grid parameters for dataset generation.
[0.9  0.95 1.  ]
[0.  0.1 0.2]
No dataset found. Generating new dataset...


  0%|          | 0/9 [00:00<?, ?it/s]

([array([-0.05854573,  1.33145936, -0.09518436,  0.09230792, -0.0055961 ,
        1.30557865, -0.15333818,  0.0106455 ]), array([-0.02655063,  1.49720514, -0.03884616, -0.26794311, -0.01034086,
        1.57509507, -0.15534615,  0.08532052]), array([-0.08483261,  1.39648956,  0.06927251, -0.06303379, -0.00534918,
        1.46612022, -0.01157227, -0.28844263]), array([-0.02701558,  1.40139623,  0.04430514,  0.12673544, -0.04403446,
        1.5176106 , -0.05029112,  0.13189061]), array([ 0.04901326,  1.56953302, -0.02689482,  0.21517471, -0.03662837,
        1.4921156 ,  0.04604117, -0.00781551])], np.int64(0), np.int64(0))


 11%|█         | 1/9 [01:05<08:42, 65.32s/it]

([array([ 0.08110507,  1.49901137, -0.10468924, -0.18588375,  0.01447787,
        1.33246303,  0.02249113, -0.35215311]), array([-0.04501116,  1.53593296, -0.12302229,  0.18243982, -0.02813674,
        1.38237965, -0.19036977, -0.20737108]), array([ 0.04325897,  1.43790498,  0.07932693,  0.2028361 , -0.05218644,
        1.4451342 , -0.0126315 , -0.32900422]), array([ 0.02744351,  1.4985489 , -0.14617154,  0.4223841 ,  0.05025   ,
        1.344582  ,  0.14283945,  0.00801327]), array([-0.03660868,  1.37079835, -0.05946316,  0.05798624, -0.01754816,
        1.56831859,  0.04480933, -0.39205217])], np.int64(0), np.int64(1))


...

([array([ 0.04015232,  1.49171262,  0.08830982,  0.60811217,  0.03204902,
        1.79538365,  0.20374795, -0.14279656]), array([-0.02601062,  1.51356987,  0.14123832, -0.03611214,  0.01298561,
        1.61776746, -0.04170505,  0.0933688 ]), array([ 0.00769126,  1.59802859,  0.02380962,  0.15091742,  0.05747868,
        1.57509685, -0.0747301 , -0.05889274]), array([-0.01769052,  1.41139274,  0.05057015,  0.31351839,  0.03024183,
        1.53054521, -0.03701215, -0.01592268]), array([ 0.08555604,  1.67590583, -0.04764115,  0.0976829 ,  0.06322711,
        1.67763073,  0.00531213, -0.09200261])], np.int64(2), np.int64(1))


 89%|████████▉ | 8/9 [07:03<00:50, 50.26s/it]

([array([-0.01261742,  1.56451222, -0.19731502,  0.38871428, -0.00317696,
        1.57999166,  0.05260194, -0.00401636]), array([ 0.04869261,  1.43970147, -0.0298189 ,  0.63513116,  0.05478278,
        1.5155953 ,  0.0215234 , -0.10493884]), array([ 0.0384457 ,  1.52174794, -0.04088092,  0.0450055 , -0.05620055,
        1.69934165, -0.01771916, -0.32608565]), array([-0.01297731,  1.3966071 , -0.02441625, -0.0434327 , -0.13806146,
        1.5954312 ,  0.15355227,  0.11424494]), array([-0.08435546,  1.7487134 , -0.05411384,  0.12771075,  0.01291343,
        1.45984982,  0.04712796,  0.05842642])], np.int64(2), np.int64(2))


100%|██████████| 9/9 [07:52<00:00, 52.49s/it]

Dataset saved to dataset.npz


In [16]:
def build_joint_fnn_model(input_shape: Tuple[int], num_loss: int, num_gamma: int) -> Model:
    num_classes = num_loss * num_gamma
    inputs = Input(shape=input_shape, name='features')
    
    # Larger or additional Dense layers
    x = Dense(512, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    # Possibly another layer if you want
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    # Final output for 400 bins
    joint_output = Dense(num_classes, activation='softmax', name='joint_output')(x)
    model = Model(inputs=inputs, outputs=joint_output)
    return model

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

model_config = PIPELINE_CONFIG["model"]
input_shape = model_config["input_shape"]

model = build_joint_fnn_model(input_shape, num_loss, num_gamma)
model.compile(
    optimizer=Adam(learning_rate=model_config["learning_rate"]),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
print("Starting model training...")

# Option 1: No validation split
history = model.fit(
    X_data, y_joint,
    epochs=model_config["epochs"],
    batch_size=model_config["batch_size"],
    validation_split=0,
    callbacks=[ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)],
    verbose=1
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features (InputLayer)           │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ joint_output (Dense)            │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 449,161 (1.71 MB)

 Trainable params: 449,161 (1.71 MB)

 Non-trainable params: 0 (0.00 B)

Starting model training...
Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.1810 - loss: 2.2053 - learning_rate: 0.0010
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0549 - loss: 2.2148 - learning_rate: 0.0010
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2062 - loss: 2.2021 - learning_rate: 0.0010
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1662 - loss: 2.1851 - learning_rate: 0.0010
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0757 - loss: 2.1776 - learning_rate: 0.0010
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1157 - loss: 2.1785 - learning_rate: 0.0010
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1157 - loss: 2.2055 - learning_rate: 0.0010
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1053 - loss: 2.2230 - learning_rate: 0.0010
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0801 - loss: 2.2023 - learning_rate: 0

In [ ]:
model_path = PIPELINE_CONFIG["save_paths"]["model"]
model.save(model_path)
print(f"Trained model saved to {model_path}")

Trained model saved to trained_model.h5
